# SQL Soda Notebook

## Introduction

This notebook is an introduction to SQL. We will be working with a SQLite dataset: The WSDR Database. From the WSDR Database, we will be working with the soda table that includes 8 columns and 1048575 rows. The soda table will allow us to create queries to produce Pandas dataframes. 

https://www.chicagobooth.edu/research/kilts/datasets/dominicks

It has already been truncated to fit on Datahub



In [ ]:
import sqlalchemy
import sqlite3
import pandas as pd
import numpy as np
import plotly.express as px
from datetime import datetime, timedelta



In [ ]:
%load_ext sql

In [ ]:
# create a SQL Alchemy connection to the database

%sql sqlite:///wsdr.db
conn = sqlite3.connect("wsdr.db")
engine = sqlalchemy.create_engine("sqlite:///wsdr.db")

## The WSDR Database

Let's load in the database by using cell magic so that we can explore the database. 

In [ ]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

From running the above cell, we see the database has 2 tables: soda, upc.

### Part 1: Exploring the Data

We want to read the data from SQL using a SQL query. When writing a SQLite database, only SQL queries are accepted.

Using SQL Alchemy - we can write SQL commands to pass into Python functions - that can yield us Pandas Dataframes

SQL queries will be written within """ in the beginning and end of the code. After the query is written, we will use pd.read_sql(query, engine) to return a Pandas dataframe of the outputted data. An outline of a written query with the corresponding pandas function is shown below.

`query = """SELECT ..."""`

`output = pd.read_sql(query, engine)`

`output`

#### What does the soda table look like? Let's create a query below to select all the values from the soda table.

In [ ]:
%sql SELECT * FROM soda LIMIT 10;



## Same Code but into Pandas
Pass the SQL query into a Pandas command and yield a Pandas DF from the query


In [ ]:
example1 = """
SELECT * FROM soda;
"""

result1 = pd.read_sql(example1, engine)
result1

#### How many unique stores are there? Write a query to display the result.

In [ ]:
%%sql
SELECT count(DISTINCT store) 
        AS total_store
FROM soda;


### Part 2: Sample Query 

#### Exercise 1: How many distinct UPCs (universal product codes) are there? Write a query to display the result.

In [ ]:
%%sql
SELECT DISTINCT upc
        AS distinct_upc
FROM soda
ORDER BY distinct_upc
LIMIT 10;

#### Exercise 2a: Display the stores and their total sales from highest to lowest. Write a query to display the result. (MOVE is the # units sold)

In [ ]:

exercise2 = """
SELECT * FROM (SELECT store, SUM(MOVE), week 
FROM soda 
GROUP BY store 
ORDER BY SUM(MOVE) DESC);
"""

res2 = pd.read_sql(exercise2, engine)
res2

#### Exercise 2c: Create a plot with the store number and their total sales of Soda.

Becuase this already in a Pandas DF we can call pandas plotting commands

In [ ]:
res2.plot.scatter(x = 'store', y = 'SUM(MOVE)')

#### Exercise 2d: What are some observations that you notice from the graph?

*insert answer here*

Possible answer: The highest number of units sold is over 700,000 while the lowest number of units sold is around 100,000.

#### Exercise 3: Display a dataframe that shows the singular store and the number of units it sold. Write a query to display the result.  Two options are lowest selling store and highest selling store


In [ ]:
# answer

exercise3 = """
SELECT * FROM (SELECT store, SUM(MOVE) 
FROM soda 
GROUP BY store 
ORDER BY SUM(MOVE) ) LIMIT 1 ;
"""

res3 = pd.read_sql(exercise3, engine)
res3

In [ ]:
exercise3 = """
SELECT STORE, SUM(MOVE) AS total_units_sold
FROM soda
GROUP BY STORE
ORDER BY total_units_sold DESC
LIMIT 1;
"""

res3 = pd.read_sql(exercise3, conn)
res3

##  What are the top 3 upc sold

In [ ]:
%%sql
SELECT soda.UPC, SUM(soda.MOVE) AS total_sales
FROM soda
JOIN upc  ON soda.UPC = upc.UPC
GROUP BY soda.UPC
ORDER BY total_sales DESC
LIMIT 3;


## There is a shortcut where you can call tables by abbreviations

In [ ]:
%%sql
SELECT s.UPC, SUM(s.MOVE) AS total_sales
FROM soda s
JOIN upc u ON s.UPC = u.UPC
GROUP BY s.UPC
ORDER BY total_sales DESC
LIMIT 3;


In [ ]:
%%sql
SELECT *
FROM upc
WHERE UPC IN ('1200000230', '1200000231', '1200000013');

## Let's combine those 2 into a subquery

In [ ]:
%%sql
SELECT *
FROM upc 
JOIN (
    SELECT soda.UPC
    FROM soda 
    GROUP BY soda.UPC
    ORDER BY SUM(soda.MOVE) DESC
    LIMIT 3
) top_upcs ON upc.UPC = top_upcs.UPC;


## 2 liter pepsi is the winner

Where is coke?

In [ ]:
%%sql
SELECT *
FROM upc
WHERE DESCRIP LIKE '%Coke%'


In [ ]:
%%sql
SELECT *
FROM upc
WHERE DESCRIP LIKE '%Coke%'
AND SIZE = '2 LT';


In [ ]:
%%sql
SELECT *
FROM upc
WHERE DESCRIP LIKE '%Coke%'
AND DESCRIP NOT LIKE '%Cherry%'
AND DESCRIP NOT LIKE '%Diet%'
AND SIZE = '2 LT';


In [ ]:
%%sql
SELECT u.*
FROM upc u
JOIN (
    SELECT s.UPC, SUM(s.MOVE) AS total_sales
    FROM soda s
    JOIN upc u ON s.UPC = u.UPC
    WHERE u.DESCRIP LIKE '%Coke%'
    GROUP BY s.UPC
    ORDER BY total_sales DESC
) AS top_selling ON u.UPC = top_selling.UPC;



In [ ]:
%%sql
SELECT u.*, top_selling.total_sales
FROM upc u
JOIN (
    SELECT s.UPC, SUM(s.MOVE) AS total_sales
    FROM soda s
    JOIN upc u ON s.UPC = u.UPC
    WHERE u.DESCRIP LIKE '%Coke%'
    GROUP BY s.UPC
    ORDER BY total_sales DESC
) AS top_selling ON u.UPC = top_selling.UPC;


## WHAT IS COKE II?
https://en.wikipedia.org/wiki/New_Coke


In [ ]:
%%sql
SELECT WEEK, SUM(MOVE) AS total_units_sold
FROM soda
GROUP BY WEEK
ORDER BY WEEK;

In [ ]:
weekly_sales_top_store_df = %sql SELECT WEEK, SUM(MOVE) AS total_units_sold FROM soda WHERE STORE = (SELECT STORE FROM (SELECT STORE FROM soda GROUP BY STORE ORDER BY SUM(MOVE) DESC LIMIT 1)) GROUP BY WEEK ORDER BY WEEK;
weekly_sales_top_store_df = weekly_sales_top_store_df.DataFrame()
weekly_sales_top_store_df

In [ ]:
weekly_sales_top_store_df.plot(x = 'WEEK', y = 'total_units_sold')

# Side Quest on Time!

In [ ]:
# Convert WEEK column to integer (in case it's stored as a string)
weekly_sales_top_store_df["WEEK"] = pd.to_numeric(weekly_sales_top_store_df["WEEK"], errors="coerce")

# Define the start date of Week 1 in the Dominick's dataset
start_date = datetime(1989, 9, 14)

# Assign correct date values based on week number
weekly_sales_top_store_df["Start_Date"] = weekly_sales_top_store_df["WEEK"].apply(
    lambda w: start_date + timedelta(weeks=w-1) if pd.notnull(w) else None
)

# Ensure Start_Date is properly formatted as a datetime type
weekly_sales_top_store_df["Start_Date"] = pd.to_datetime(weekly_sales_top_store_df["Start_Date"])

# Sort by Start_Date to ensure a correct time series order
weekly_sales_top_store_df = weekly_sales_top_store_df.sort_values("Start_Date")
weekly_sales_top_store_df

In [ ]:
weekly_sales_top_store_df.plot(x = 'Start_Date', y = 'total_units_sold')

In [ ]:
# plot weekly sales for the top store in plotly 
fig = px.line(weekly_sales_top_store_df, x='Start_Date', y='total_units_sold', title='Weekly Sales for Top Store')
fig.show()